<a href="https://colab.research.google.com/github/kotharisanjana/CMPE258_DeepLearning_Spring2023/blob/main/Assignment_4/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Imports

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_tuner
from kerastuner.tuners import RandomSearch

<ipython-input-2-35815b0da3de>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Download  dataset
dataset, info = tfds.load('horses_or_humans', with_info=True, as_supervised=True)

# Split the dataset into training and test sets
train_dataset = dataset['train']
test_dataset = dataset['test']

print(info)

tfds.core.DatasetInfo(
    name='horses_or_humans',
    full_name='horses_or_humans/3.0.0',
    description="""
    A large set of images of horses and humans.
    """,
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    data_path='/root/tensorflow_datasets/horses_or_humans/3.0.0',
    file_format=tfrecord,
    download_size=153.59 MiB,
    dataset_size=153.53 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=256, num_shards=1>,
        'train': <SplitInfo num_examples=1027, num_shards=2>,
    },
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or Humans Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/horses-or-humans-dataset"
    }""",
)


In [4]:
batch_size = 32
epochs = 5

In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    image = tf.image.resize(image, (300, 300))
    return image, label

In [6]:
# Create train and test sets

train_dataset = train_dataset.map(preprocess).batch(batch_size)
X_train = []
y_train = []

for images, labels in train_dataset.as_numpy_iterator():
    X_train.append(images)
    y_train.append(labels)

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)


test_dataset = test_dataset.map(preprocess).batch(batch_size)
X_test = []
y_test = []

for images, labels in test_dataset.as_numpy_iterator():
    X_test.append(images)
    y_test.append(labels)

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [7]:
X_train.shape, X_test.shape

((1027, 300, 300, 3), (256, 300, 300, 3))

In [8]:
# Create validation set from training set

train_val_split = int(0.9*X_train.shape[0])
print(train_val_split)

X_val, y_val = X_train[train_val_split:], y_train[train_val_split:]
X_train, y_train = X_train[:train_val_split], y_train[:train_val_split]

924


In [9]:
X_train.shape, X_val.shape, X_test.shape

((924, 300, 300, 3), (103, 300, 300, 3), (256, 300, 300, 3))

In [10]:
model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300,300,3)),
            tf.keras.layers.MaxPooling2D((2,2)),
            tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2,2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(20, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
            ])

# Train model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/5
29/29 [==============================] - 14s 160ms/step - loss: 0.5928 - accuracy: 0.7652 - val_loss: 0.1657 - val_accuracy: 0.9223
Epoch 2/5
29/29 [==============================] - 2s 62ms/step - loss: 0.0624 - accuracy: 0.9827 - val_loss: 0.0174 - val_accuracy: 1.0000
Epoch 3/5
29/29 [==============================] - 2s 64ms/step - loss: 0.0100 - accuracy: 0.9978 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 4/5
29/29 [==============================] - 2s 72ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 5/5
8/8 [==============================] - 0s 41ms/step - loss: 2.6664 - accuracy: 0.6836
Test accuracy: 0.68359375


### Hyperparameter tuning

Search over the learning rate, number of filters in the convolutional layers, and number of neurons in the fully connected layers.

In [22]:
def create_model(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(hp.Int('filters1', 32, 96, step=32), (3,3), activation='relu', input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.MaxPooling2D((2,2)))
  model.add(tf.keras.layers.Conv2D(hp.Int('filters2', 32, 96, step=32), (3,3), activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D((2,2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(hp.Int('units1', 10, 50, step=32), activation='relu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Train    
  model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])), loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [23]:
# Define tuner

tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=epochs,
    executions_per_trial=2,
    directory='keras_tuner',
)

In [24]:
# Search for the best hyperparameters

tuner.search(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

In [25]:
# Print best hyperparameters

best_hyperparams = tuner.get_best_hyperparameters()[0]
print('Best hyperparameters:')
print(f'learning_rate = {best_hyperparams.get("learning_rate")}')
print(f'filters1 = {best_hyperparams.get("filters1")}')
print(f'filters2 = {best_hyperparams.get("filters2")}')
print(f'units1 = {best_hyperparams.get("units1")}')

Best hyperparameters:
learning_rate = 0.001
filters1 = 64
filters2 = 64
units1 = 20


In [26]:
# Create a new model with the best hyperparameters

model = tuner.hypermodel.build(best_hyperparams)

model.compile(optimizer=tf.keras.optimizers.Adam(best_hyperparams.get('learning_rate')), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the best hyperparameters
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

# Evaluate the model on the test set with the best hyperparameters
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy with hyperparameter tuning:', test_acc)

Epoch 1/5
29/29 [==============================] - 10s 209ms/step - loss: 0.7944 - accuracy: 0.7424 - val_loss: 0.2648 - val_accuracy: 0.8932
Epoch 2/5
29/29 [==============================] - 3s 107ms/step - loss: 0.1432 - accuracy: 0.9578 - val_loss: 0.1107 - val_accuracy: 0.9515
Epoch 3/5
29/29 [==============================] - 3s 106ms/step - loss: 0.0285 - accuracy: 0.9913 - val_loss: 0.0242 - val_accuracy: 0.9903
Epoch 4/5
29/29 [==============================] - 3s 114ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 5/5
8/8 [==============================] - 0s 47ms/step - loss: 2.4477 - accuracy: 0.7266
Test accuracy with hyperparameter tuning: 0.7265625


After hyper parameter tuning, test accuracy increases to 72.6% from 68.3%

### Data augmentation + Hyperparameter tuning

In [27]:
# Data augmentation pipeline

data_augmentation = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                         tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                         tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)])

In [28]:
def create_model_(hp):
    model = tf.keras.Sequential([
        data_augmentation,
        tf.keras.layers.Conv2D(hp.Choice('filters1', values=[32, 64]), kernel_size=3, activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(hp.Choice('filters2', values=[32, 64]), kernel_size=3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),
        tf.keras. layers.Flatten(),
        tf.keras.layers.Dense(hp.Choice('units1', values=[20, 50, 100]), activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [29]:
# Define the hyperparameter search space

tuner_search = RandomSearch(
    create_model_,
    objective='val_accuracy',
    max_trials=epochs,
    executions_per_trial=2,
    directory='tuner_dir',
)

In [30]:
# Perform the hyperparameter search

tuner_search.search(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Trial 5 Complete [00h 02m 51s]
val_accuracy: 0.6990291178226471

Best val_accuracy So Far: 0.9223300814628601
Total elapsed time: 00h 13m 26s


In [31]:
# Get the best hyperparameters after data augmentation and hyperparamter tuning

best_hp = tuner_search.get_best_hyperparameters(num_trials=1)[0]
print(f'Best hyperparameters:\n{best_hp.values}\n')

Best hyperparameters:
{'filters1': 64, 'filters2': 64, 'units1': 100, 'learning_rate': 0.0001}



In [32]:
# Build the model with the best hyperparameters

model = tuner_search.hypermodel.build(best_hp)

In [33]:
# Train the model

model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/5
29/29 [==============================] - 19s 547ms/step - loss: 1.8272 - accuracy: 0.5628 - val_loss: 0.5610 - val_accuracy: 0.6893
Epoch 2/5
29/29 [==============================] - 16s 544ms/step - loss: 0.5146 - accuracy: 0.7619 - val_loss: 0.4166 - val_accuracy: 0.8252
Epoch 3/5
29/29 [==============================] - 16s 547ms/step - loss: 0.4081 - accuracy: 0.8258 - val_loss: 0.3289 - val_accuracy: 0.8641
Epoch 4/5
29/29 [==============================] - 16s 555ms/step - loss: 0.3194 - accuracy: 0.8647 - val_loss: 0.2195 - val_accuracy: 0.8738
Epoch 5/5
29/29 [==============================] - 16s 542ms/step - loss: 0.3094 - accuracy: 0.8820 - val_loss: 0.1848 - val_accuracy: 0.9320


In [34]:
# Evaluate the model on the test set

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy with hyperparameter tuning and data augmentation: {test_acc}')

8/8 [==============================] - 1s 72ms/step - loss: 1.7272 - accuracy: 0.5352
Test accuracy with hyperparameter tuning and data augmentation: 0.53515625


Adding data augmentation reduced accuract on the drastically but it reduced the loss also in comparison to base model and model with only hyperparameter tuning.